# Entrega 1
- Descripción del sistema computacional:
- Pregunta de estudio: 
- Tipos de datos a recolectar:

# Entrega 2
- Metodología de evaluación de desempeño:
    (Monitoreo, benchmarking o redes de colas)
- Diseño del estudio (y primeros resultados):

# Entrega 3
Informe final con
- Recoleccion de Datos
- Resultados
> con interpretacion
- Conclusiones

$$\text{Lista Nueva lista de Data}$$

<img src="newData/list.png" alt="Girl in a jacket">

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [2]:
def readData(filename, sep=",", parse_dates=[]):
    data = pd.read_csv(filename, sep=sep)    
    for col in parse_dates:
        data[col] = pd.to_datetime(data[col], errors="coerce")
    return data
    
def hCumsum(data):
    columns = data.columns[1:]
    data[columns] = data[columns].cumsum(axis=1)
    return data, columns

In [3]:
# 1. Read and clean data

# 1.1 WebData
archivos = [
    "CPU Utilization.csv", #0
    "GPU Utilization.csv", #1
    "GPU Used Memory.csv", #2
    "Memory.csv",          #3
    "Patagon - Power.csv", #4
]

data = []
cols_ix = [] 
for i, a in enumerate(archivos):
    d, c = hCumsum(readData(a, parse_dates=["Time"]))
    data.append(d)
    cols_ix.append(c)
    
# 1.2 PatagonData
monitoreo = readData("monitoreo.csv", sep="|", parse_dates=['Start', 'End'])

##### Cantidad de Nulls por data (donde hayan)
Universo: 15412
- "Unnamed: 19": todos jj
- "MinCPU": 924
- "MaxVMSize": 924
- "AveVMSize"
- "MaxRSS": 924
- "AveRSS": 924
- "MaxDiskWrite": 924
- "AveDiskWrite": 924
- "MaxDiskRead": 924
- "AveDiskRead": 924
- "ReqTRES": 10364

In [4]:
filtrado = [
    'Unnamed: 19',
    'ReqTRES'
]

In [5]:
filtered_monitoreo = monitoreo.copy()
filtered_monitoreo.drop([0, 1, 2], axis=0, inplace=True)
filtered_monitoreo.drop(filtrado, axis=1, inplace=True)

In [6]:
filtered_monitoreo["ReqMem"] = filtered_monitoreo.ReqMem.str.extract('(\d+)')
filtered_monitoreo["AveRSS"] = filtered_monitoreo.AveRSS.str.extract('(\d+)')
filtered_monitoreo["MaxRSS"] = filtered_monitoreo.MaxRSS.str.extract('(\d+)')
filtered_monitoreo["AveVMSize"] = filtered_monitoreo.AveVMSize.str.extract('(\d+)')
filtered_monitoreo["MaxVMSize"] = filtered_monitoreo.MaxVMSize.str.extract('(\d+)')

In [7]:
filtered_monitoreo.fillna(0, inplace=True)

In [8]:
filtered_monitoreo["ReqMem"] = filtered_monitoreo["ReqMem"].astype(np.int64)
filtered_monitoreo["AveRSS"] = filtered_monitoreo["AveRSS"].astype(np.int64)
filtered_monitoreo["MaxRSS"] = filtered_monitoreo["MaxRSS"].astype(np.int64)
filtered_monitoreo["AveVMSize"] = filtered_monitoreo["AveVMSize"].astype(np.int64)
filtered_monitoreo["MaxVMSize"] = filtered_monitoreo["MaxVMSize"].astype(np.int64)

In [9]:
filtered_monitoreo

,State,Start,End,Elapsed,ReqMem,AveDiskRead,MaxDiskRead,AveDiskWrite,MaxDiskWrite,AveRSS,MaxRSS,AveVMSize,MaxVMSize,ReqCPUS,CPUTime,CPUTimeRAW,MinCPU,TotalCPU
3,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0,0,0,0,0,0,0,0,32,00:00:32,32,0,00:00.017
4,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0,0,0,0,1260,1260,207696,207696,32,00:00:32,32,00:00:00,00:00.014
5,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0,0,0,0,816,816,273752,273752,32,00:00:32,32,00:00:00,00:00.002
6,FAILED,2021-05-01 16:34:16,2021-05-01 16:34:17,00:00:01,100000,0,0,0,0,0,0,0,0,32,00:00:32,32,0,00:00.016
7,FAILED,2021-05-01 16:34:16,2021-05-01 16:34:17,00:00:01,100000,0,0,0,0,1264,1264,207696,207696,32,00:00:32,32,00:00:00,00:00.013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15407,FAILED,2021-07-05 09:33:11,2021-07-05 09:33:12,00:00:01,0,0,0,0,0,864,864,273752,273752,8,00:00:08,8,00:00:00,00:00.004
15408,COMPLETED,2021-07-05 09:33:12,2021-07-05 09:33:14,00:00:02,0,0,0,0,0,856,856,273752,273752,8,00:00:16,16,00:00:00,00:00.821
15409,COMPLETED,2021-07-05 09:33:14,2021-07-05 09:33:15,00:00:01,0,0,0,0,0,860,860,273752,273752,8,00:00:08,8,00:00:00,00:00.776
15410,RUNNING,2021-07-05 09:33:15,0,07:12:40,0,0,0,0,0,0,0,0,0,8,2-09:41:20,207680,0,00:00:00


In [10]:
filtered_monitoreo.describe()

,ReqMem,AveRSS,MaxRSS,AveVMSize,MaxVMSize,ReqCPUS,CPUTimeRAW
count,15409.000000,1.540900e+04,1.540900e+04,1.540900e+04,1.540900e+04,15409.000000,1.540900e+04
mean,20741.991304,3.080200e+06,3.079731e+06,2.030566e+07,8.367880e+06,7.389578,1.070038e+04
std,63496.763751,1.001432e+07,1.001433e+07,1.077791e+09,2.423262e+07,10.191706,1.440460e+05
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000e+00
25%,0.000000,8.560000e+02,8.560000e+02,2.737520e+05,2.737520e+05,1.000000,1.600000e+01
50%,0.000000,8.600000e+02,8.600000e+02,2.737520e+05,2.737520e+05,8.000000,8.800000e+01
75%,65556.000000,1.880000e+04,1.879600e+04,3.403160e+05,3.403160e+05,8.000000,2.800000e+02
max,958444.000000,1.097190e+08,1.097190e+08,1.136220e+11,2.812504e+08,648.000000,8.575200e+06


In [ ]:
completos = filtered_monitoreo[filtered_monitoreo["State"] == "COMPLETED"]
fallidos = filtered_monitoreo[filtered_monitoreo["State"] == "FAILED"]
corriendos = filtered_monitoreo[filtered_monitoreo["State"] == "RUNNING"]
cancelados = filtered_monitoreo[filtered_monitoreo["State"] == "CANCELLED"]
# dataframe[dataframe['Percentage'] > 70]

In [ ]:
plt.figure(figsize=(15, 6), tight_layout=True)
# plt.bar([1,2,3], [1,2,3])

In [ ]:
for i in range(len(data)):
    plt.figure(figsize=(15, 10))
    plt.plot(data[i]["Time"], data[i][cols_ix[i]])
    plt.savefig("{}.png".format(archivos[i]), bbox_inches='tight')